In [1]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots


import pandas as pd
import numpy as np

### Run this to bypass the two next sections (get unblocked fev1 and preprocessd data)

In [2]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx_and_heighest_obs_per_id")

# Get unblocked FEV1

In [3]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")

In [ ]:
df = df.drop(columns=["PEF", "ecPEF (L/s)", "PEF (L/s)"])
df = df.dropna(subset=["FEV1", "O2 Saturation", "FEF2575"])
df["ecFEF2575%ecFEV1"] = df["FEF2575"] / df["FEV1"] * 100
print(df.shape)
print("IDs: ", df["ID"].nunique())
df.head()

(41272, 16)
IDs:  353


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1
0,101,2019-01-25,1.31,97.0,0.54,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374
1,101,2019-01-26,1.31,98.0,0.57,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450
2,101,2019-01-27,1.31,96.0,0.67,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038
3,101,2019-01-28,1.30,96.0,0.69,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.010470,98.816157,53.076923
4,101,2019-01-29,1.28,98.0,0.60,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,35.456463,100.874827,46.875000


In [4]:
# Add unblocked FEV1
def get_unblocked_fev1_for_ID(df):
    # Find index where ecFEV1 is max
    idx = df["ecFEV1"].idxmax()
    df["Max ecFEV1"] = df.loc[idx, "ecFEV1"]
    df["Max ecFEF2575"] = df.loc[idx, "ecFEF2575"]
    return df


df = (
    df.groupby("ID")
    .apply(get_unblocked_fev1_for_ID)
    .drop(columns=["ID"])
    .reset_index()
    .drop(columns=["level_1"])
)

In [5]:
# How often is max ecFEV1 obtained at the same time as max ecFEF2575?
# Filter idx where max ecFEV1 = ecFEV1
idx_max = df["Max ecFEV1"] == df["ecFEV1"]
df_tmp = df[idx_max].copy()
df_tmp["Drop from max ecFEF2575"] = np.abs(
    df_tmp["Max ecFEF2575"] - df_tmp["ecFEF2575"]
)

(df_tmp["Drop from max ecFEF2575"] == 0).value_counts()
# Most of the time, max ecFEV1 is obtained at the same time as max ecFEF2575

Drop from max ecFEF2575
True     669
False    271
Name: count, dtype: int64

In [6]:
df.columns

Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'ecFEV1',
       'ecFEF2575', 'Sex', 'Height', 'Age', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy', 'ecFEF2575%ecFEV1', 'Max ecFEV1',
       'Max ecFEF2575'],
      dtype='object')

# Preprocess breathe Data

In [7]:
(HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat, ecFEF2575prctecFEV1) = (
    var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        160, 40, "Male"
    )
)

In [9]:
# Applied get_bin_for_value to all inputs and updated excel file
df[f"idx {ecFEV1.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["ecFEV1"], ecFEV1)[1], axis=1
)
df[f"idx {ecFEF2575prctecFEV1.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["ecFEF2575%ecFEV1"], ecFEF2575prctecFEV1)[1],
    axis=1,
)
df[f"idx {O2Sat.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["O2 Saturation"], O2Sat)[1], axis=1
)

In [10]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,Max ecFEV1,Max ecFEF2575,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%)
0,101,2019-01-25,1.31,97.0,0.54,1.31,0.67,Male,173.0,53,...,97.150104,36.287474,36.287474,99.845492,41.221374,1.79,1.14,26,20,47
1,101,2019-01-26,1.31,98.0,0.57,1.31,0.67,Male,173.0,53,...,97.150104,36.287474,36.287474,100.874827,43.511450,1.79,1.14,26,21,48
2,101,2019-01-27,1.31,96.0,0.67,1.31,0.69,Male,173.0,53,...,97.150104,36.287474,36.287474,98.816157,51.145038,1.79,1.14,26,25,46
3,101,2019-01-28,1.30,96.0,0.69,1.31,0.69,Male,173.0,53,...,97.150104,36.287474,36.010470,98.816157,53.076923,1.79,1.14,26,26,46
4,101,2019-01-29,1.28,98.0,0.60,1.30,0.69,Male,173.0,53,...,97.150104,36.010470,35.456463,100.874827,46.875000,1.79,1.14,26,23,48


In [11]:
# df.to_excel("BR_O2_FEV1_FEF2575_with_idx.xlsx", index=False)

# Use approximate inference (slicing)

In [55]:
df_for_ID.iloc[0]

ID                                    101
Date Recorded                  2019-01-25
FEV1                                 1.31
O2 Saturation                        97.0
FEF2575                              0.54
ecFEV1                               1.31
ecFEF2575                            0.67
Sex                                  Male
Height                              173.0
Age                                    53
Predicted FEV1                   3.610061
Healthy O2 Saturation           97.150104
ecFEV1 % Predicted              36.287474
FEV1 % Predicted                36.287474
O2 Saturation % Healthy         99.845492
ecFEF2575%ecFEV1                41.221374
Max ecFEV1                           1.79
Max ecFEF2575                        1.14
idx ecFEV1 (L)                         26
idx ecFEF25-75 % ecFEV1 (%)            20
idx O2 saturation (%)                  47
Name: 0, dtype: object

In [67]:
idx = df_for_ID.sort_values(
    by=["ecFEV1", "ecFEF2575", "O2 Saturation"], ascending=False
).index[0]

df_for_ID.loc[idx]

# pd.concat([df_for_ID.iloc[0], df_for_ID.iloc[idx]], axis=1)

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,Max ecFEV1,Max ecFEF2575,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%)
0,101,2019-01-25,1.31,97.0,0.54,1.31,0.67,Male,173.0,53,...,97.150104,36.287474,36.287474,99.845492,41.221374,1.79,1.14,26,20,47
1576,101,2023-07-31,1.79,98.0,1.14,1.79,1.15,Male,173.0,53,...,97.150104,49.583647,49.583647,100.874827,63.687151,1.79,1.14,35,31,48


In [5]:
(HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat, ecFEF2575prctecFEV1) = (
    var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        160, 40, "Male"
    )
)

key_hfev1 = f"['{ecFEV1.name}', '{HFEV1.name}', '{AR.name}'] -> {HFEV1.name}"
key_ho2sat = f"['{O2SatFFA.name}', '{HO2Sat.name}', '{AR.name}'] -> {HO2Sat.name}"
HFEV1.set_factor_node_key(key_hfev1)
HO2Sat.set_factor_node_key(key_ho2sat)

vars = [AR, IA]
shared_vars = [HFEV1, HO2Sat]
obs_vars = [ecFEV1.name, O2Sat.name]
# obs_vars = [ecFEV1.name, O2Sat.name, ecFEF2575prctecFEV1.name]


def infer_and_plot_for_id(df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-8):
    for shared_var in shared_vars:
        shared_var.reset()
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"#datapoints: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    (
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(height, age, sex)

    # Given an ID, get the data which maximises ecFEV1, then ecFEF2575, then O2 Saturation
    idx_unblocked_FEV1 = df_for_ID.sort_values(
        by=["ecFEV1", "ecFEF2575", "O2 Saturation"], ascending=False
    ).index[0]

    for i, _ in df_for_ID[0:1].iterrows():
        df_two_days = df_for_ID.iloc[[i, idx_unblocked_FEV1]]

        df_query_res, df_res_before_convergence, shared_vars_final = (
            slicing.query_across_days(
                df_two_days, inf_alg, shared_vars, vars, obs_vars, diff_threshold
            )
        )

    return df_query_res, df_res_before_convergence, shared_vars_final


df_for_ID = df[df["ID"] == "101"]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-2
)

# df.groupby("ID").apply(
#     lambda df_for_ID: infer_and_plot_for_id(
#         df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-6
#     )
# )

In [109]:
df_query_res

,Day,Airway resistance (%),Inactive alveoli (%),Healthy FEV1 (L),Healthy O2 saturation (%)
0,2019-01-25,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7285264891441958, 0.2334362304876964, 0.035...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2023-07-31,"[1.149777912333138e-05, 3.459197720488003e-05,...","[0.8111381584522424, 0.17105992825686503, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [112]:
# 4 rows: prior, posterior for healthy var, then ar day 1, ar day 3
fig = make_subplots(rows=4, cols=2)
ar_day_1 = df_query_res.loc[0, AR.name]
ar_day_2 = df_query_res.loc[1, AR.name]
ia_day_1 = df_query_res.loc[0, IA.name]
ia_day_2 = df_query_res.loc[1, IA.name]
hfev1 = df_query_res.loc[0, HFEV1.name]
ho2sat = df_query_res.loc[0, HO2Sat.name]

# Priors
ih.plot_histogram(
    fig, HFEV1, HFEV1.cpt, HFEV1.a, HFEV1.b, 1, 1, HFEV1.name + " prior", "#0072b2"
)
ih.plot_histogram(
    fig, HO2Sat, HO2Sat.cpt, HO2Sat.a, HO2Sat.b, 1, 2, HO2Sat.name + " prior", "#009e73"
)
# Posteriors
ih.plot_histogram(
    fig,
    HFEV1,
    hfev1,
    HFEV1.a,
    HFEV1.b,
    2,
    1,
    HFEV1.name + " shared posterior",
    "#0072b2",
)
ih.plot_histogram(
    fig,
    HO2Sat,
    ho2sat,
    HO2Sat.a,
    HO2Sat.b,
    2,
    2,
    HO2Sat.name + " shared posterior",
    "#009e73",
)
# AR
ih.plot_histogram(
    fig,
    AR,
    ar_day_1,
    AR.a,
    AR.b,
    3,
    1,
    AR.name + " day 1",
    colour="#d55e00",
)
ih.plot_histogram(
    fig,
    AR,
    ar_day_2,
    AR.a,
    AR.b,
    4,
    1,
    AR.name + " day 2",
    colour="#d55e00",
)
# IA
ih.plot_histogram(
    fig,
    IA,
    ia_day_1,
    IA.a,
    IA.b,
    3,
    2,
    IA.name + " day 1",
    "#d55e00",
)
ih.plot_histogram(
    fig,
    IA,
    ia_day_2,
    IA.a,
    IA.b,
    4,
    2,
    IA.name + " day 2",
    "#d55e00",
)
fig.update_xaxes(title_standoff=5)

fig.update_layout(
    title="hello", width=800, height=600, font=dict(size=8), showlegend=False
)

fig.show()

In [81]:
print(
    f"HFEV1 max diff between 2 days: {max(df_query_res.loc[0, HFEV1.name] - df_query_res.loc[1, HFEV1.name]):.2g}"
)
print(
    f"HO2Sat max diff between 2 days: {max(df_query_res.loc[0, HO2Sat.name] - df_query_res.loc[1, HO2Sat.name]):.2g}"
)
# Stop criteria: difference of the last HFEV1 inferred between two epochs is les than a threshold (1e-6)

HFEV1 max diff between 2 days: 2.2e-07
HO2Sat max diff between 2 days: 1.6e-06


# Use exact inference

In [1]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx_and_heighest_obs_per_id")
df_for_ID = df[df["ID"] == "101"]
idx_unblocked_FEV1 = df_for_ID.sort_values(
        by=["ecFEV1", "ecFEF2575", "O2 Saturation"], ascending=False
    ).index[0]

df_for_ID.iloc[[0, idx_unblocked_FEV1]]

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,Max ecFEV1,Max ecFEF2575,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.67,Male,173.0,53,...,97.150104,36.287474,36.287474,99.845492,41.221374,1.79,1.14,26,20,47
1576,101,2023-07-31,1.79,98,1.14,1.79,1.15,Male,173.0,53,...,97.150104,49.583647,49.583647,100.874827,63.687151,1.79,1.14,35,31,48


In [2]:
height = df_for_ID.Height.iloc[0]
age = df_for_ID.Age.iloc[0]
sex = df_for_ID.Sex.iloc[0]
(
    model,
    inf_alg,
    HFEV1,
    HO2Sat,
    ecFEV1,
    AR,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    # ecFEF2575prctecFEV1,
    ecFEV1_2,
    AR_2,
    O2SatFFA_2,
    IA_2,
    UO2Sat_2,
    O2Sat_2,
    # ecFEF2575prctecFEV1_2,
) = mb.o2_sat_fev1_fef2575_two_days_model(height, age, sex)

In [3]:
# vars = [HFEV1, HO2Sat, AR, IA, AR_2, IA_2]
vars = [AR]
obs_vars = [[ecFEV1, 1.31], [O2Sat, 97], [ecFEV1_2, 1.79], [O2Sat_2, 98]]

res = ih.infer(
    inference_alg=inf_alg,
    variables=vars,
    evidences=obs_vars
)

/Users/tristan.trebaol/Desktop/PhD/Code/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


In [7]:
res

{'Healthy FEV1 (L)': <DiscreteFactor representing phi(Healthy FEV1 (L):100) at 0x7fd7d1e6b970>,
 'Healthy O2 saturation (%)': <DiscreteFactor representing phi(Healthy O2 saturation (%):20) at 0x7fd7d1e6ad70>,
 'Airway resistance (%)': <DiscreteFactor representing phi(Airway resistance (%):45) at 0x7fd7d1e6afe0>,
 'Inactive alveoli (%)': <DiscreteFactor representing phi(Inactive alveoli (%):30) at 0x7fd7d1e6ad10>,
 'Airway resistance (%) day 2': <DiscreteFactor representing phi(Airway resistance (%) day 2:45) at 0x7fd7d1e6b4f0>}